<a href="https://colab.research.google.com/github/alisonnnnn88/programming_language/blob/main/HW6_AI_%E5%AD%B8%E7%BF%92%E7%AD%86%E8%A8%98%E7%B3%BB%E7%B5%B1(Gradio%E7%89%88%E6%9C%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -------------------------------
# 安裝必要套件 (Colab 執行一次)
# -------------------------------
!apt install tesseract-ocr -y
!pip install pdfplumber pillow gspread google-auth google-auth-oauthlib google-auth-httplib2 gradio -q

# -------------------------------
# 匯入模組
# -------------------------------
import pdfplumber
import requests, json, time
import gspread
from google.colab import auth
from google.auth import default
from datetime import datetime
import pytz
import gradio as gr

# -------------------------------
# Google Sheet 認證 (Colab)
# -------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1h75idXVO6GMosL5bPUF-G76MqxOziHu1tJieRtpdkP4/edit?usp=sharing"
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet_name = "學習筆記"
try:
    ws = sh.worksheet(worksheet_name)
except gspread.exceptions.WorksheetNotFound:
    ws = sh.add_worksheet(title=worksheet_name, rows=100, cols=20)

# 初始化標題列（如果空的話）
if ws.get_all_values() == []:
    ws.append_row(["時間_TW", "PDF名稱", "頁碼", "文字前500字", "摘要", "題目"])

# -------------------------------
# PDF 文字抓取 (單頁純文字)
# -------------------------------
def extract_text_single_page(pdf_path, page_num=1):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if page_num < 1 or page_num > len(pdf.pages):
                return f"⚠️ PDF 總共有 {len(pdf.pages)} 頁，輸入頁數超出範圍"
            text = pdf.pages[page_num-1].extract_text()
            if not text:
                return "⚠️ 該頁無可抓取文字 (可能是掃描 PDF)"
            return text.strip()
    except Exception as e:
        return f"❌ PDF 擷取失敗：{e}"

# -------------------------------
# REST 呼叫 Gemini (安全版) - single request with retries
# -------------------------------
def call_gemini_rest(prompt, api_key, model_name="models/gemini-2.5-flash", timeout=60, max_retries=3):
    """
    使用 REST API 呼叫 Gemini generateContent。返回生成文字或錯誤字串。
    """
    endpoint = f"https://generativelanguage.googleapis.com/v1beta/{model_name}:generateContent"
    headers = {
        "Content-Type": "application/json",
        "x-goog-api-key": api_key
    }
    data = {"contents": [{"parts": [{"text": prompt}]}]}
    for attempt in range(1, max_retries+1):
        try:
            resp = requests.post(endpoint, headers=headers, json=data, timeout=timeout)
            if resp.status_code == 200:
                j = resp.json()
                # 取第一候選的所有 parts 串接
                parts = j.get("candidates", [])[0]["content"]["parts"]
                return "".join(p.get("text","") for p in parts)
            else:
                # 非 200，列出訊息再重試
                err = f"HTTP {resp.status_code}: {resp.text}"
                print(f"⚠️ 呼叫失敗 (attempt {attempt}): {err}")
        except requests.exceptions.Timeout:
            print(f"⚠️ 呼叫逾時 (attempt {attempt})")
        except Exception as e:
            print(f"⚠️ 呼叫錯誤 (attempt {attempt}): {e}")
        time.sleep(2 ** attempt)  # backoff: 2,4,8 ...
    return "❌ 生成失敗，請稍後再試。"

# -------------------------------
# 封裝：分段摘要 + 題目（避免一次過大）
# -------------------------------
def generate_summary_and_quiz_rest(text, api_key, model_name="models/gemini-2.5-flash"):
    """
    對 text 做分段摘要（每段交給模型），最後再生成題目一次。
    回傳 (summary_text, quiz_text)；如果失敗，回傳錯誤字串。
    """
    # 限制總長，避免超大內容（可依需求調整）
    max_total = 4000
    if len(text) > max_total:
        text = text[:max_total]

    # 決定 chunk size（字元），chunk 太小會呼叫太多次，太大容易 timeout
    chunk_size = 1200
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    summary_parts = []
    for i, chunk in enumerate(chunks, 1):
        # prompt：請模型針對該 chunk 列出 3 點摘要（保持簡潔）
        prompt = f"請把以下內容用條列整理 3 點重點（每點一句話）：\n\n{chunk}"
        summary_part = call_gemini_rest(prompt, api_key, model_name=model_name, timeout=60, max_retries=2)
        if summary_part.startswith("❌"):
            # 若某段失敗，記錄失敗並停止或繼續視需求
            summary_parts.append("[段落生成失敗]")
        else:
            summary_parts.append(summary_part.strip())
    # 合併各段摘要並壓縮成 5 行內總結
    merged = "\n".join(f"- {s}" for s in summary_parts)
    compress_prompt = f"請把下面的條列內容再濃縮成 5 行左右的整體摘要（每行一句）：\n\n{merged}"
    final_summary = call_gemini_rest(compress_prompt, api_key, model_name=model_name, timeout=60, max_retries=2)

    # 生成題目（根據最初 text 的前 1200 字）
    quiz_prompt = f"根據以下內容，出 3 題簡答題並附標準答案：\n\n{text[:1200]}"
    final_quiz = call_gemini_rest(quiz_prompt, api_key, model_name=model_name, timeout=60, max_retries=2)

    return final_summary, final_quiz

# -------------------------------
# Gradio 主流程
# -------------------------------
def process_pdf_rest(pdf_file, page_num_str, api_key, model_choice):
    # immediate checks
    if not pdf_file:
        yield "⚠️ 請上傳 PDF", "", ""
        return
    if not api_key:
        yield "⚠️ 請輸入 API Key", "", ""
        return
    try:
        page_num = int(page_num_str)
    except:
        yield "⚠️ 頁數輸入錯誤", "", ""
        return

    pdf_path = pdf_file.name
    text = extract_text_single_page(pdf_path, page_num)
    if text.startswith("⚠️") or text.startswith("❌"):
        yield text, "", ""
        return

    # 進度：開始分段摘要
    yield "🔍 開始生成摘要（會分段以降低超時風險）", "", ""
    summary, quiz = generate_summary_and_quiz_rest(text, api_key, model_name=model_choice)

    # 若生成失敗會回傳錯誤字串
    yield text[:500], summary, quiz

    # 寫入 Google Sheet（最後）
    tz = pytz.timezone("Asia/Taipei")
    now_str = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    try:
        ws.append_row([now_str, pdf_path, page_num, text[:500], summary, quiz])
    except Exception as e:
        yield f"⚠️ Sheet 回寫失敗：{e}", summary, quiz

# -------------------------------
# Gradio 介面
# -------------------------------
# 你可在下拉選 model（根據你列出模型清單選擇）
MODEL_OPTIONS = ["models/gemini-2.5-flash", "models/gemini-2.5-pro", "models/gemini-2.0-flash"]

with gr.Blocks() as demo:
    gr.Markdown("## PDF → 摘要 + 題目（REST API 版，穩定）")

    with gr.Row():
        pdf_input = gr.File(label="上傳 PDF", file_types=[".pdf"])
        page_input = gr.Textbox(label="頁碼", placeholder="例如：1", value="1")
        api_input = gr.Textbox(label="Gemini API Key", type="password", placeholder="請輸入你的 API Key")
        model_select = gr.Dropdown(MODEL_OPTIONS, value=MODEL_OPTIONS[0], label="模型")

    text_output = gr.Textbox(label="文字前500字", lines=6)
    summary_output = gr.Textbox(label="摘要", lines=8)
    quiz_output = gr.Textbox(label="題目", lines=8)

    run_btn = gr.Button("開始生成")
    run_btn.click(process_pdf_rest, inputs=[pdf_input, page_input, api_input, model_select],
                  outputs=[text_output, summary_output, quiz_output])

demo.launch()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19bf9d2de39ad97e48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
